In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.keras.applications import MobileNetV3Small
import time
keras = tf.keras
layers = tf.keras.layers
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0  # normalize to [0,1] range
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\0\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\0\test'
checkpoint_path = r'C:\Users\24306\Desktop\mobilenetv3\0\mobilenetv3.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[13.0, 35.0, 100.0, 95.0, 86.0, 13.0, 45.0, 96.0, 100.0, 47.0, 40.0, 47.0, 96.0, 100.0, 95.0, 32.0, 95.0, 37.0, 7.0, 96.0, 13.0, 32.0, 45.0, 16.0, 7.0, 32.0, 23.0, 95.0, 13.0, 100.0, 37.0, 80.0, 7.0, 45.0, 47.0, 32.0, 66.0, 100.0, 66.0, 39.0, 10.0, 16.0, 16.0, 40.0, 37.0, 55.0, 32.0, 95.0, 16.0, 86.0, 47.0, 37.0, 40.0, 80.0, 95.0, 100.0, 13.0, 40.0, 35.0, 95.0, 80.0, 66.0, 86.0, 7.0, 45.0, 86.0, 4.0, 66.0, 66.0, 47.0, 4.0, 96.0, 100.0, 23.0, 30.0, 86.0, 52.0, 37.0, 35.0, 47.0, 10.0, 86.0, 7.0, 10.0, 86.0, 13.0, 32.0, 40.0, 93.0, 32.0, 40.0, 37.0, 100.0, 47.0, 23.0, 93.0, 55.0, 95.0, 37.0, 16.0, 55.0, 100.0, 47.0, 10.0, 13.0, 4.0, 45.0, 86.0, 37.0, 66.0, 16.0, 84.0, 84.0, 86.0, 84.0, 39.0, 37.0, 100.0, 100.0, 45.0, 80.0, 84.0, 80.0, 95.0, 86.0, 47.0, 13.0, 39.0, 95.0, 80.0, 84.0, 66.0, 47.0, 37.0, 13.0, 96.0, 47.0, 66.0, 66.0, 35.0, 30.0, 35.0, 4.0, 45.0, 100.0, 7.0, 16.0, 47.0, 37.0, 16.0, 86.0, 30.0, 30.0, 35.0, 80.0, 100.0, 80.0, 96.0, 84.0, 96.0, 100.0, 96.0, 96.0, 100.0, 95.0, 66.0

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
MobileNetV3 = MobileNetV3Small(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
MobileNetV3.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
x = MobileNetV3(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
MobilenetV3small (Functional (None, 1, 1, 1024)        1529968   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
out_weight (Dense)           (None, 1)                 1025      
Total params: 1,530,993
Trainable params: 1,518,881
Non-trainable params: 12,112
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
np.savetxt('mobilenetv3_loss.txt',history.history['loss'])
#np.savetxt('DenseNet201_acc.txt',history.history['acc'])
np.savetxt('mobilenetv3_val_loss.txt',history.history['val_loss'])
#np.savetxt('DenseNet201_val_acc.txt',history.history['val_acc'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Fri Mar 31 20:56:28 2023
Epoch 1/300
591/591 [==============================] - 47s 57ms/step - loss: 45.6092 - val_loss: 3134.9080
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\0\mobilenetv3.ckpt\assets


C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/300
591/591 [==============================] - 31s 52ms/step - loss: 3.0908 - val_loss: 2964.0554
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\0\mobilenetv3.ckpt\assets


C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 3/300
591/591 [==============================] - 29s 49ms/step - loss: 1.8152 - val_loss: 2371.5728
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\0\mobilenetv3.ckpt\assets


...

Epoch 232/300
591/591 [==============================] - 59s 99ms/step - loss: 0.1578 - val_loss: 0.1072
Epoch 233/300
591/591 [==============================] - 59s 99ms/step - loss: 0.1589 - val_loss: 0.1142
Epoch 234/300
591/591 [==============================] - 58s 99ms/step - loss: 0.1530 - val_loss: 0.1107
Epoch 235/300
591/591 [==============================] - 59s 99ms/step - loss: 0.1584 - val_loss: 0.1240
Epoch 236/300
591/591 [==============================] - 58s 98ms/step - loss: 0.1504 - val_loss: 0.1288
Epoch 237/300
591/591 [==============================] - 58s 99ms/step - loss: 0.1542 - val_loss: 0.1227
Epoch 238/300
591/591 [==============================] - 58s 99ms/step - loss: 0.1472 - val_loss: 0.1607
Epoch 239/300
591/591 [==============================] - 58s 98ms/step - loss: 0.1676 - val_loss: 0.1246
Epoch 240/300
591/591 [==============================] - 58s 98ms/step - loss: 0.1522 - val_loss: 0.1170
Epoch 241/300
591/591 [==============================] 

C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 278/300
591/591 [==============================] - 62s 104ms/step - loss: 0.1401 - val_loss: 0.1205
Epoch 279/300
591/591 [==============================] - 61s 103ms/step - loss: 0.1497 - val_loss: 0.1166
Epoch 280/300
591/591 [==============================] - 61s 103ms/step - loss: 0.1556 - val_loss: 0.1171
Epoch 281/300
591/591 [==============================] - 60s 102ms/step - loss: 0.1526 - val_loss: 0.1219
Epoch 282/300
591/591 [==============================] - 61s 102ms/step - loss: 0.1489 - val_loss: 0.1249
Epoch 283/300
591/591 [==============================] - 61s 102ms/step - loss: 0.1557 - val_loss: 0.1053
Epoch 284/300
591/591 [==============================] - 61s 103ms/step - loss: 0.1627 - val_loss: 0.1028
INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\mobilenetv3\0\mobilenetv3.ckpt\assets


C:\Users\24306\.conda\envs\keras\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 285/300
591/591 [==============================] - 61s 103ms/step - loss: 0.1504 - val_loss: 0.1036
Epoch 286/300
591/591 [==============================] - 60s 102ms/step - loss: 0.1563 - val_loss: 0.1174
Epoch 287/300
591/591 [==============================] - 60s 102ms/step - loss: 0.1556 - val_loss: 0.1162
Epoch 288/300
591/591 [==============================] - 60s 102ms/step - loss: 0.1467 - val_loss: 0.1166
Epoch 289/300
591/591 [==============================] - 60s 102ms/step - loss: 0.1737 - val_loss: 0.1199
Epoch 290/300
591/591 [==============================] - 60s 101ms/step - loss: 0.1669 - val_loss: 0.1184
Epoch 291/300
591/591 [==============================] - 60s 102ms/step - loss: 0.1851 - val_loss: 0.1100
Epoch 292/300
591/591 [==============================] - 60s 102ms/step - loss: 0.1517 - val_loss: 0.1098
Epoch 293/300
591/591 [==============================] - 61s 102ms/step - loss: 0.1437 - val_loss: 0.1129
Epoch 294/300
591/591 [=======================